In [ ]:
# 1. 加载要索引的数据。  
# 使用WebBaseLoader加载网页数据  
from langchain_community.document_loaders import WebBaseLoader  
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")  
docs = loader.load()  
  
# 2. 文档分片，并通过向量存储构建文档索引   
from langchain_openai import OpenAIEmbeddings  
embeddings = OpenAIEmbeddings()  
# FAISS：轻量化的本地存储  
from langchain_community.vectorstores import FAISS  
from langchain_text_splitters import RecursiveCharacterTextSplitter  
#文档分片
text_splitter = RecursiveCharacterTextSplitter()  
documents = text_splitter.split_documents(docs)  
# 分片数据添加索引
vector = FAISS.from_documents(documents, embeddings)  
  

# 索引转为检索器 
from langchain.chains import create_retrieval_chain  
retriever = vector.as_retriever()  

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI  
llm = ChatOpenAI()  

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

# 首先，我们需要一个可以传递给LLM来生成此搜索查询的提示
# 能够搜索对话的
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "根据上面的对话，生成一个搜索查询来获取与对话相关的信息")
])
# 接收对话历史并返回文档的chain
# retriever：上面创建的数据索引
retriever_chain = create_history_aware_retriever(
												 llm, # 大模型
												 retriever, # 文档检索器
												 prompt) # 有历史对话并能基于历史对话来生成搜索查询（能够补全下面的问题？）

from langchain_openai import ChatOpenAI  
llm = ChatOpenAI()  

# 由聊天记录生成的prompt，
prompt = ChatPromptTemplate.from_messages([
    ("system", "根据下面的上下文回答用户的问题：\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
# 创建一个chain，将对话列表传递给大模型
document_chain = create_stuff_documents_chain(llm, prompt)
# 创建一个检索链，用于检索文档（和对话）并将它们传递给下一个步骤。
# 能够检索历史对话，即能够基于对话上下文，继续回答
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# 封装的历史对话：之后通过Memory来实现？
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})